In [9]:
!pip3 install scikit-multilearn
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import LabelPowerset

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
def load_domain():
    return pd.read_excel('./Dataset/Domain_Classification_Data/Synthetic User Stories.xlsx')

In [12]:
def ml_tasks_bin():
    dataset = pd.read_excel("./Dataset/ML_Tasks_Classification_Data/Synthetic User Stories.xlsx", header=None)
    labels = pd.read_excel("./Dataset/ML_Tasks_Classification_Data/Keyword labelled.xlsx", header=None)
    labels[2] = labels[2].apply(lambda x: x.lower())
    categories_column = []
    for row in labels.iterrows():
        current_labels = []
        for label in row[1][3:]:
            if isinstance(label, str):
                current_labels.append(label.lower())
        categories_column.append(current_labels)
    labels["Categories array"] = categories_column
    target = []
    counter = 0
    for row in dataset.iterrows():
        target.append(labels[labels[2]==row[1]["Machine Learning Task"].lower()]["Categories array"].values[0])
        counter += 1
    dataset["Target"] = target
    dataset['Target'] = dataset['Target'].apply(lambda x: ast.literal_eval(str(x)))

    multilabel = MultiLabelBinarizer()
    y = multilabel.fit_transform(dataset['Target'])
    return pd.DataFrame(y, columns=multilabel.classes_)